<a href="https://www.kaggle.com/code/samithsachidanandan/text-classification-using-tensorflow?scriptVersionId=278439912" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import Libraries

In [1]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [2]:
import tensorflow as tf 
from tensorflow import keras 
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt
import time 

2025-11-15 19:46:18.074805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763235978.412874      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763235978.499200      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### laod the dataset 

In [3]:
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')


### Basic Statistics 

In [4]:
df.shape

(7613, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [7]:
df.dtypes

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

In [8]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [9]:
df.duplicated().sum()

0

In [10]:
print((df.target ==1).sum())
print((df.target ==0).sum())

3271
4342


In [11]:
# Preprocessing
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")
for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
t
@bbcmtd Wholesale Markets ablaze 


In [13]:
df["text"] = df.text.map(remove_URL) # map(lambda x: remove_URL(x))
df["text"] = df.text.map(remove_punct)

In [14]:
!pip install nltk

In [15]:
# remove stopwords
# pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them 
# as the result of a search query.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
stop

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [17]:
df['texts'] = df.text.map(remove_stopwords)

In [18]:
df.text

0       Our Deeds are the Reason of this earthquake Ma...
1                   Forest fire near La Ronge Sask Canada
2       All residents asked to shelter in place are be...
3       13000 people receive wildfires evacuation orde...
4       Just got sent this photo from Ruby Alaska as s...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    ariaahrary TheTawniest The out of control wild...
7610                M194 0104 UTC5km S of Volcano Hawaii 
7611    Police investigating after an ebike collided w...
7612    The Latest More Homes Razed by Northern Califo...
Name: text, Length: 7613, dtype: object

In [19]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df.text)

In [20]:

len(counter)

22225

In [21]:
counter

Counter({'Our': 26,
         'Deeds': 2,
         'are': 347,
         'the': 2595,
         'Reason': 5,
         'of': 1731,
         'this': 347,
         'earthquake': 26,
         'May': 16,
         'ALLAH': 1,
         'Forgive': 1,
         'us': 66,
         'all': 191,
         'Forest': 38,
         'fire': 174,
         'near': 50,
         'La': 11,
         'Ronge': 1,
         'Sask': 1,
         'Canada': 7,
         'All': 54,
         'residents': 5,
         'asked': 9,
         'to': 1815,
         'shelter': 5,
         'in': 1777,
         'place': 26,
         'being': 68,
         'notified': 1,
         'by': 472,
         'officers': 8,
         'No': 60,
         'other': 33,
         'evacuation': 35,
         'or': 181,
         'orders': 11,
         'expected': 14,
         '13000': 4,
         'people': 164,
         'receive': 2,
         'wildfires': 10,
         'California': 113,
         'Just': 49,
         'got': 104,
         'sent': 12,
        

In [22]:

counter.most_common(5)

[('the', 2595), ('a', 1874), ('to', 1815), ('in', 1777), ('of', 1731)]

In [23]:
num_unique_words = len(counter)

In [24]:
# Split dataset into training and validation set
train_size = int(df.shape[0] * 0.8)

train_df = df[:train_size]
val_df = df[train_size:]

# split text and labels
train_sentences = train_df.text.to_numpy()
train_labels = train_df.target.to_numpy()
val_sentences = val_df.text.to_numpy()
val_labels = val_df.target.to_numpy()

In [25]:

train_sentences.shape, val_sentences.shape

((6090,), (1523,))

In [26]:

# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [27]:
# each word has unique index
word_index = tokenizer.word_index

In [28]:

word_index

{'the': 1,
 'a': 2,
 'to': 3,
 'in': 4,
 'of': 5,
 'and': 6,
 'i': 7,
 'is': 8,
 'for': 9,
 'on': 10,
 'you': 11,
 'my': 12,
 'with': 13,
 'it': 14,
 'that': 15,
 'by': 16,
 'at': 17,
 'this': 18,
 'are': 19,
 'from': 20,
 'be': 21,
 'was': 22,
 'have': 23,
 'up': 24,
 'me': 25,
 'like': 26,
 'just': 27,
 'as': 28,
 'so': 29,
 'amp': 30,
 'but': 31,
 'not': 32,
 'fire': 33,
 'im': 34,
 'your': 35,
 'out': 36,
 'no': 37,
 'will': 38,
 'an': 39,
 'after': 40,
 'all': 41,
 'its': 42,
 'when': 43,
 'if': 44,
 'we': 45,
 'get': 46,
 'has': 47,
 'via': 48,
 'new': 49,
 'now': 50,
 'more': 51,
 'about': 52,
 'or': 53,
 'people': 54,
 'he': 55,
 'what': 56,
 'over': 57,
 'they': 58,
 'news': 59,
 'dont': 60,
 'emergency': 61,
 'do': 62,
 'how': 63,
 'one': 64,
 'been': 65,
 '2': 66,
 'us': 67,
 'into': 68,
 'there': 69,
 'were': 70,
 'can': 71,
 'video': 72,
 'disaster': 73,
 'burning': 74,
 'body': 75,
 'than': 76,
 'her': 77,
 'would': 78,
 'buildings': 79,
 'police': 80,
 'who': 81,
 'his':

In [29]:

train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [30]:

print(train_sentences[10:15])
print(train_sequences[10:15])

['Three people died from the heat wave so far'
 'Haha South Tampa is getting flooded hah WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK flooding'
 'raining flooding Florida TampaBay Tampa 18 or 19 days Ive lost count '
 'Flood in Bago Myanmar We arrived Bago'
 'Damage to school bus on 80 in multi car crash BREAKING ']
[[624, 54, 492, 20, 1, 240, 389, 29, 509], [860, 569, 2370, 8, 221, 2371, 2937, 625, 2, 716, 7, 276, 4, 569, 2370, 56, 185, 7, 277, 62, 56, 185, 7, 277, 62, 5809, 197], [2938, 197, 2003, 5810, 2370, 1399, 53, 1566, 626, 347, 752, 2939], [175, 4, 3866, 717, 45, 1567, 3866], [191, 3, 167, 431, 10, 3867, 4, 3868, 122, 83, 406]]


In [31]:
# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 20

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((6090, 20), (1523, 20))

In [32]:

train_padded[10]

array([624,  54, 492,  20,   1, 240, 389,  29, 509,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [33]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

Three people died from the heat wave so far
[624, 54, 492, 20, 1, 240, 389, 29, 509]
[624  54 492  20   1 240 389  29 509   0   0   0   0   0   0   0   0   0
   0   0]


In [34]:

# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [35]:

reverse_word_index

{1: 'the',
 2: 'a',
 3: 'to',
 4: 'in',
 5: 'of',
 6: 'and',
 7: 'i',
 8: 'is',
 9: 'for',
 10: 'on',
 11: 'you',
 12: 'my',
 13: 'with',
 14: 'it',
 15: 'that',
 16: 'by',
 17: 'at',
 18: 'this',
 19: 'are',
 20: 'from',
 21: 'be',
 22: 'was',
 23: 'have',
 24: 'up',
 25: 'me',
 26: 'like',
 27: 'just',
 28: 'as',
 29: 'so',
 30: 'amp',
 31: 'but',
 32: 'not',
 33: 'fire',
 34: 'im',
 35: 'your',
 36: 'out',
 37: 'no',
 38: 'will',
 39: 'an',
 40: 'after',
 41: 'all',
 42: 'its',
 43: 'when',
 44: 'if',
 45: 'we',
 46: 'get',
 47: 'has',
 48: 'via',
 49: 'new',
 50: 'now',
 51: 'more',
 52: 'about',
 53: 'or',
 54: 'people',
 55: 'he',
 56: 'what',
 57: 'over',
 58: 'they',
 59: 'news',
 60: 'dont',
 61: 'emergency',
 62: 'do',
 63: 'how',
 64: 'one',
 65: 'been',
 66: '2',
 67: 'us',
 68: 'into',
 69: 'there',
 70: 'were',
 71: 'can',
 72: 'video',
 73: 'disaster',
 74: 'burning',
 75: 'body',
 76: 'than',
 77: 'her',
 78: 'would',
 79: 'buildings',
 80: 'police',
 81: 'who',
 82: 'h

In [36]:

def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [37]:
decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)

[624, 54, 492, 20, 1, 240, 389, 29, 509]
three people died from the heat wave so far


In [38]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))
model.add(layers.SpatialDropout1D(0.2))
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-11-15 19:46:48.584526: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC()]
)

In [40]:

model.fit(train_padded, train_labels, epochs=30, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/30
191/191 - 21s - 112ms/step - accuracy: 0.6770 - auc: 0.7308 - loss: 0.5912 - val_accuracy: 0.5680 - val_auc: 0.8303 - val_loss: 0.6614
Epoch 2/30
191/191 - 6s - 32ms/step - accuracy: 0.8399 - auc: 0.9010 - loss: 0.3831 - val_accuracy: 0.7360 - val_auc: 0.8324 - val_loss: 0.5841
Epoch 3/30
191/191 - 6s - 32ms/step - accuracy: 0.9090 - auc: 0.9573 - loss: 0.2518 - val_accuracy: 0.7485 - val_auc: 0.8241 - val_loss: 0.5112
Epoch 4/30
191/191 - 6s - 33ms/step - accuracy: 0.9404 - auc: 0.9787 - loss: 0.1741 - val_accuracy: 0.7459 - val_auc: 0.8052 - val_loss: 0.6856
Epoch 5/30
191/191 - 6s - 32ms/step - accuracy: 0.9514 - auc: 0.9845 - loss: 0.1467 - val_accuracy: 0.7229 - val_auc: 0.7847 - val_loss: 0.7990
Epoch 6/30
191/191 - 6s - 32ms/step - accuracy: 0.9550 - auc: 0.9896 - loss: 0.1213 - val_accuracy: 0.7288 - val_auc: 0.7860 - val_loss: 0.8215
Epoch 7/30
191/191 - 6s - 32ms/step - accuracy: 0.9634 - auc: 0.9938 - loss: 0.0953 - val_accuracy: 0.7111 - val_auc: 0.7857 - val_los

In [41]:
predictions = model.predict(train_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]

191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


In [42]:
print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])

['Three people died from the heat wave so far'
 'Haha South Tampa is getting flooded hah WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK flooding'
 'raining flooding Florida TampaBay Tampa 18 or 19 days Ive lost count '
 'Flood in Bago Myanmar We arrived Bago'
 'Damage to school bus on 80 in multi car crash BREAKING ' 'Whats up man'
 'I love fruits' 'Summer is lovely' 'My car is so fast'
 'What a goooooooaaaaaal']
[1 1 1 1 1 0 0 0 0 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
